# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import re
import gensim
import copy
import random
import uts

# Создадим функцию загрузки исходных данных

In [2]:
def load_articles(path):
    
    data =  pd.DataFrame(columns=['text', 'label'])
    
    introductions = os.listdir(path + 'introductions/')
    for file in introductions:
        f = open(path + 'introductions/' + file)
        tmp_text = f.read()
        data = data.append({'text': tmp_text, 'label':0}, ignore_index = True)
    
    introductions = os.listdir(path + 'main_parts/')
    for file in introductions:
        f = open(path + 'main_parts/' + file)
        tmp_text = f.read()
        data = data.append({'text': tmp_text, 'label':1}, ignore_index = True)
    
    introductions = os.listdir(path + 'conclusions/')
    for file in introductions:
        f = open(path + 'conclusions/' + file)
        tmp_text = f.read()
        data = data.append({'text': tmp_text, 'label':2}, ignore_index = True)
    
    return data
    

# Создадим функцию очистки данных

In [3]:
def clear_text(text):
    text = text.lower()
    text = re.sub(r'\\d+:\\d+(:\\d+)?', ' время ', text)
    text = re.sub(r'\\b\\w\\b', ' ', text)
    text = re.sub(r'\\b\\d+\\b', ' цифра ', text)
    text = re.sub("^\s+|\n|\r|\s+$", ' ', text) # регулярка удаляет пробелы в начале и конце строки, а также символы переноса
    text = re.sub("([\[]).*?([\]])", "", text) # регулярка удаляет квадратные скобки
    text = re.sub(r'[\\W]+', ' ', text)
    text = re.sub('[!@#$(),^+=*><№«//•»;]', '', text)
    return text

# Создадим функцию очистки статей

In [4]:
def clear_articles(data):
    for i in range(data.shape[0]):
        data.iloc[i,0] = clear_text(data.iloc[i,0])
    return data

# Собираем все вместе

In [5]:
data = load_articles('./data_for_train_classifire/')
data = clear_articles(data)

данные для обучения классификатора получены

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.preprocessing import sequence
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [7]:
model_vect = gensim.models.KeyedVectors.load("./model.model")

In [8]:
vect_of_sent = np.zeros((data.shape[0],300))

In [9]:
for id, s in enumerate(data.text):
        v = np.zeros(300)
        t = 0
        for word in s.split():
            try:
                v += model_vect[word]
                t+=1
            except KeyError:
                pass
        vect_of_sent[id] = v/t

In [10]:
X = vect_of_sent
Y = data.label

In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1)

In [12]:
model = Sequential()
model.add(Dense(4, input_dim=300, activation='elu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train, Y_train, batch_size=128, epochs=200,
          validation_split=0.1)

Train on 433 samples, validate on 49 samples
Epoch 1/200
433/433 [==============================] - 0s 286us/step - loss: 1.2080 - acc: 0.3372 - val_loss: 1.0686 - val_acc: 0.4898
Epoch 2/200
433/433 [==============================] - 0s 40us/step - loss: 1.0683 - acc: 0.4157 - val_loss: 1.0400 - val_acc: 0.4898
Epoch 3/200
433/433 [==============================] - 0s 38us/step - loss: 1.0568 - acc: 0.4319 - val_loss: 1.0379 - val_acc: 0.4082
Epoch 4/200
433/433 [==============================] - 0s 25us/step - loss: 1.0367 - acc: 0.4180 - val_loss: 1.0815 - val_acc: 0.4286
Epoch 5/200
433/433 [==============================] - 0s 21us/step - loss: 1.0336 - acc: 0.4365 - val_loss: 1.0191 - val_acc: 0.4898
Epoch 6/200
433/433 [==============================] - 0s 34us/step - loss: 1.0119 - acc: 0.4388 - val_loss: 1.0364 - val_acc: 0.4286
Epoch 7/200
433/433 [==============================] - 0s 32us/step - loss: 1.0157 - acc: 0.4619 - val_loss: 0.9799 - val_acc: 0.4694
Epoch 8/200
433/

433/433 [==============================] - 0s 46us/step - loss: 0.7045 - acc: 0.7090 - val_loss: 0.7589 - val_acc: 0.7755
Epoch 62/200
433/433 [==============================] - 0s 29us/step - loss: 0.6780 - acc: 0.7367 - val_loss: 0.7433 - val_acc: 0.7143
Epoch 63/200
433/433 [==============================] - 0s 35us/step - loss: 0.6769 - acc: 0.7344 - val_loss: 0.6988 - val_acc: 0.6735
Epoch 64/200
433/433 [==============================] - 0s 29us/step - loss: 0.6724 - acc: 0.7298 - val_loss: 0.7130 - val_acc: 0.6531
Epoch 65/200
433/433 [==============================] - 0s 30us/step - loss: 0.6720 - acc: 0.7390 - val_loss: 0.7469 - val_acc: 0.6122
Epoch 66/200
433/433 [==============================] - 0s 45us/step - loss: 0.7086 - acc: 0.6651 - val_loss: 0.7600 - val_acc: 0.7551
Epoch 67/200
433/433 [==============================] - 0s 32us/step - loss: 0.6661 - acc: 0.7506 - val_loss: 0.7215 - val_acc: 0.6122
Epoch 68/200
433/433 [==============================] - 0s 36us/step

433/433 [==============================] - 0s 27us/step - loss: 0.5551 - acc: 0.7875 - val_loss: 0.6596 - val_acc: 0.6735
Epoch 122/200
433/433 [==============================] - 0s 30us/step - loss: 0.5525 - acc: 0.7806 - val_loss: 0.6655 - val_acc: 0.6939
Epoch 123/200
433/433 [==============================] - 0s 25us/step - loss: 0.5489 - acc: 0.8014 - val_loss: 0.6464 - val_acc: 0.6531
Epoch 124/200
433/433 [==============================] - 0s 27us/step - loss: 0.5785 - acc: 0.7667 - val_loss: 0.6390 - val_acc: 0.6327
Epoch 125/200
433/433 [==============================] - 0s 28us/step - loss: 0.5455 - acc: 0.7968 - val_loss: 0.6427 - val_acc: 0.6531
Epoch 126/200
433/433 [==============================] - 0s 30us/step - loss: 0.5410 - acc: 0.7921 - val_loss: 0.6295 - val_acc: 0.7347
Epoch 127/200
433/433 [==============================] - 0s 27us/step - loss: 0.5438 - acc: 0.8037 - val_loss: 0.7875 - val_acc: 0.6939
Epoch 128/200
433/433 [==============================] - 0s 21

433/433 [==============================] - 0s 30us/step - loss: 0.4752 - acc: 0.8176 - val_loss: 0.6132 - val_acc: 0.6939
Epoch 182/200
433/433 [==============================] - 0s 20us/step - loss: 0.4984 - acc: 0.8199 - val_loss: 0.6284 - val_acc: 0.6939
Epoch 183/200
433/433 [==============================] - 0s 37us/step - loss: 0.4733 - acc: 0.8106 - val_loss: 0.6557 - val_acc: 0.6327
Epoch 184/200
433/433 [==============================] - 0s 30us/step - loss: 0.4692 - acc: 0.8268 - val_loss: 0.6063 - val_acc: 0.6735
Epoch 185/200
433/433 [==============================] - 0s 24us/step - loss: 0.4653 - acc: 0.8314 - val_loss: 0.6700 - val_acc: 0.6531
Epoch 186/200
433/433 [==============================] - 0s 26us/step - loss: 0.4870 - acc: 0.8222 - val_loss: 0.6139 - val_acc: 0.6735
Epoch 187/200
433/433 [==============================] - 0s 33us/step - loss: 0.4799 - acc: 0.8245 - val_loss: 0.6574 - val_acc: 0.6735
Epoch 188/200
433/433 [==============================] - 0s 31

In [14]:
accr = model.evaluate(X_test,Y_test)

54/54 [==============================] - 0s 60us/step


In [15]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.948
  Accuracy: 0.685


In [16]:
predicted = model.predict_classes(X_test)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(list(Y_test), predicted))

              precision    recall  f1-score   support

           0       0.82      0.45      0.58        20
           1       0.43      1.00      0.60         6
           2       0.76      0.79      0.77        28

   micro avg       0.69      0.69      0.69        54
   macro avg       0.67      0.75      0.65        54
weighted avg       0.74      0.69      0.68        54



# Считаем, что классификатор получен

# Начинаем рассматривать кластеризацию

In [19]:
import nltk

In [20]:
def text2list(text):
    text = text.lower()
    text = re.sub(r'\\d+:\\d+(:\\d+)?', ' время ', text)
    text = re.sub(r'\\b\\w\\b', ' ', text)
    text = re.sub(r'\\b\\d+\\b', ' цифра ', text)
    text = re.sub("^\s+|\n|\r|\s+$", ' ', text) # регулярка удаляет пробелы в начале и конце строки, а также символы переноса
    text = re.sub("([\[]).*?([\]])", "", text) # регулярка удаляет квадратные скобки
    text = re.sub(r'[\\W]+', ' ', text)
    text = re.sub('[!@#$(),^+=*><№«//•»;]', '', text)
    sentences = nltk.sent_tokenize(text)
    sentences_with_filters = list()
    for sent in sentences:
        if len(sent) > 20:
            sentences_with_filters.append(sent)
    return sentences_with_filters

In [21]:
def get_sentences_and_y(text, label_flag):
    sentences = text2list(text)
    y_labels = np.zeros(len(sentences))
    if label_flag:
        y_labels[-1] = 1
    return sentences, y_labels

In [22]:
def json_to_list_of_sentences_and_y_label(file_name):
    f = open(file_name)
    data = json.load(f)
    introduction_sentences, y_labels_introduction = get_sentences_and_y(data['Введение'], True)
    main_part_sentences, y_labels_main_part = get_sentences_and_y(data['Обзор литературы']+data['Методология']+data['Результаты'], True)
    findings_sentences, y_labels_findings = get_sentences_and_y(data['Выводы'], True)
    total_list_of_sentences = list()
    total_list_of_sentences.append(introduction_sentences)
    total_list_of_sentences.append(main_part_sentences)
    total_list_of_sentences.append(findings_sentences)
    y_labels_total = np.array([])
    y_labels_total = np.append(y_labels_total, y_labels_introduction)
    y_labels_total = np.append(y_labels_total, y_labels_main_part)
    y_labels_total = np.append(y_labels_total, y_labels_findings)
    
    sen = list()
    for p in total_list_of_sentences:
        for s in p:
            sen.append(s)
    
    return sen, y_labels_total

In [23]:
def text2vect(text):
    v = np.zeros(300)
    t = 0
    for s in text:
        for word in s.split():
            try:
                v += model_vect[word]
                t+=1
            except KeyError:
                pass
    return v/t

In [24]:
def predict_p(border_1, border_2, sen):
    
    text_1 = list()
    
    for i1 in range(border_1+1):
        text_1.append(sen[i1])
    text_2 = list()
    for i2 in range(border_1+2, border_2+1):
        text_2.append(sen[i2])
    
    v1 = text2vect(text_1)
    v2 = text2vect(text_2)
    
    p1 = model.predict(v1.reshape(1, 300))[0][0]
    p2 = model.predict(v2.reshape(1, 300))[0][1]
    p = p1 * p2
    
    
    return p, border_1
    


In [25]:
def do_iter(border_1, border_2, sen):
    
    p1, fb_1 = predict_p(border_1, border_2, sen)
    
    return p1

In [26]:
def predict_p_2(border_1, border_2, sen):
    
    text_1 = list()
    
    for i1 in range(border_1+1, border_2+1):
        text_1.append(sen[i1])
    text_2 = list()
    for i2 in range(border_2+1, len(sen)):
        text_2.append(sen[i2])
    
    v1 = text2vect(text_1)
    v2 = text2vect(text_2)
    
    p1 = model.predict(v1.reshape(1, 300))[0][1]
    p2 = model.predict(v2.reshape(1, 300))[0][2]
    p = p1 * p2
    
    return p, border_2
    


In [27]:
def do_iter_2(border_1, border_2, sen):
    
    p2, fb_1 = predict_p(border_1, border_2, sen)
    
    return p2

In [28]:
def find_borders_new(path):
    sen, lables = json_to_list_of_sentences_and_y_label(path)
    #border_1 = random.randint(, round(len(sen)/3))
    #border_2 = random.randint(round(len(sen)/2), len(sen))
    #border_1 = round(len(sen)/4)
    #border_2 = round(len(sen)/4) * 3
    
    border_1 = round(len(sen)/3)
    border_2 = round(len(sen)/3) * 2
    
    p_begin = do_iter(border_1, border_2, sen)
    
    p_max = copy.copy(p_begin)
    border_1_find = copy.copy(border_1)
    
    for i in range(25):
        border_1_tmp = random.randint(2, border_2)
        p = do_iter(border_1_tmp, border_2, sen)
        if p > p_max:
            border_1_find = copy.copy(border_1_tmp)
    
    p_max = 0
    border_2_find = copy.copy(border_2)
    
    for i in range(25):
        border_2_tmp = random.randint(border_2+1, len(sen)-1)
        p = do_iter_2(border_1_find, border_2_tmp, sen)
        if p > p_max:
            border_2_find = copy.copy(border_2_tmp)
            
    c99_model = uts.C99()
    hyp_c99 = c99_model.segment(sen)
    string_c99 = ''
    for i in hyp_c99:
        if i == 0:
            string_c99+='1'
        else:
            string_c99+='0'
    
    topictilling_model = uts.TextTiling()
    hyp_topic_tilling = topictilling_model.segment(sen)
    string_topic_tilling = ''
    for i in hyp_topic_tilling:
        if i == 0:
            string_topic_tilling+='1'
        else:
            string_topic_tilling+='0'
    
    
    hyp = np.zeros(len(lables))
    hyp[border_1_find] = 1
    hyp[border_2_find] = 1
    hyp[len(sen)-1] = 1
    string_hyp = ''
    for i in hyp:
        string_hyp+=str(int(i))
    string_lables = ''
    for i in lables:
        string_lables += str(int(i))
    return string_hyp, string_lables, string_c99, string_topic_tilling
    
    

In [29]:
articles_for_test = os.listdir('./articles_for_test/')

global_hyp = ''
global_lables = ''
global_string_c99 = ''
global_topic_tilling = ''

for article in articles_for_test:
    hyp, tru, hyp_c99, hyp_topic_tilling = find_borders_new('./articles_for_test/' + article)
    global_hyp += hyp
    global_lables += tru
    global_string_c99 += hyp_c99
    global_topic_tilling += hyp_topic_tilling

/home/fr1lance/Документы/python_36/lib64/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [31]:
from nltk.metrics.segmentation import pk
from nltk.metrics.segmentation import windowdiff

### Pk метрика предлагаемого алгоритма

In [44]:
pk(str(global_lables), str(global_hyp), k=12, boundary='1')*100

27.028252001812962

### Pk метрика алгоритма C99

In [45]:
pk(str(global_lables), str(global_string_c99), k=12, boundary='1')*100

61.27813869164527

### Pk метрика алгоритма TextTiling

In [46]:
pk(str(global_lables), str(global_topic_tilling), k=12, boundary='1')*100

61.27813869164527

### windowdiff метрика предалагаемого алгоритма

In [47]:
windowdiff(str(global_lables), str(global_hyp), k=12, boundary='1')*100

37.02976280404895

### windowdiff метрика алгоритма C99

In [54]:
windowdiff(str(global_lables), str(global_string_c99), k=12, boundary='1')*100

100.0

### windowdiff метрика алгоритма TextTiling

In [51]:
windowdiff(str(global_lables), str(global_topic_tilling), k=13, boundary='1')*100

100.0

In [42]:
number_of_sentences = list()
for text in data.text.tolist():
    sentences = text2list(text)
    number_of_sentences.append(len(sentences))

In [43]:
print(np.mean(number_of_sentences))

12.875
